In [20]:
import numpy as np
#Catalogue mass and locations of components
# as arrays [mass, x, y, z]

#Components
bus = (210, 0.0, 0.0, 0.0)
cameras = (115, 0.0, 0.0, 0.75)
panel1 = (11, 1.4, 0.0, 0.0)
panel2 = (11, -1.4, 0.0, 0.0)
antenna = (5, 0.33, 0.56, 0.0)

#Components array
comps = [bus, cameras, panel1, panel2, antenna]


In [21]:
#Calculate Center of Mass
# weighted_mass = 0.0
total_mass = 0.0
weighted_pos = np.zeros(3)
for comp in comps:
    m = comp[0]
    r = np.array(comp[1:])
    weighted_pos += m * r
    total_mass += m

r_cm = weighted_pos / total_mass
print("Center of mass: ", r_cm)
print("Total mass: ", total_mass)


Center of mass:  [0.0046875  0.00795455 0.24502841]
Total mass:  352.0


In [22]:
#Calculate moment of inertias of each component

#Bus
m_bus = bus[0]
R_bus = 0.55
H_bus = 0.85

Ibus_xx = 1/12*m_bus*(3*R_bus**2 + H_bus**2)
Ibus_yy = 1/12*m_bus*(3*R_bus**2 + H_bus**2)
Ibus_zz = 1/2*m_bus*R_bus**2

#Camera structure
m_cam = cameras[0]
R_cam = 0.6
H_cam = 0.65

Icam_xx = 1/12*m_cam*(3*R_cam**2 + H_cam**2)
Icam_yy = 1/12*m_cam*(3*R_cam**2 + H_cam**2)
Icam_zz = 1/2*m_cam*R_cam**2

#Panels
m_pan = panel1[0]
w_pan = 1.1
d_pan =0.01
h_pan = 0.89

Ipan1_xx = 1/12*m_pan*(h_pan**2 + d_pan**2)
Ipan1_yy = 1/12*m_pan*(d_pan**2 + w_pan**2) 
Ipan1_zz = 1/12*m_pan*(h_pan**2 + w_pan**2)

Ipan2_xx = 1/12*m_pan*(h_pan**2 + d_pan**2)
Ipan2_yy = 1/12*m_pan*(d_pan**2 + w_pan**2) 
Ipan2_zz = 1/12*m_pan*(h_pan**2 + w_pan**2)

#Antenna
m_ant = antenna[0]
R_ant = 0.35
H_ant = 0.2

Iant_xx = 1/12*m_ant*(3*R_ant**2 + H_ant**2)
Iant_yy = 1/12*m_ant*(3*R_ant**2 + H_ant**2)
Iant_zz = 1/2*m_ant*R_ant**2


In [23]:
#Store total I of each component
I_bus = np.diag([Ibus_xx, Ibus_yy, Ibus_zz])
I_cam = np.diag([Icam_xx, Icam_yy, Icam_zz])
I_pan1 = np.diag([Ipan1_xx, Ipan1_yy, Ipan1_zz])
I_pan2 = np.diag([Ipan2_xx, Ipan2_yy, Ipan2_zz])
I_ant = np.diag([Iant_xx, Iant_yy, Iant_zz])

I_centroids = [I_bus, I_cam, I_pan1, I_pan2, I_ant]

In [24]:
#Compute total inertia matrix about r_cm
I_total = np.zeros((3,3))
I_3 = np.eye(3)

for i, (m, x, y, z) in enumerate(comps):
    r_i = np.array([x, y, z])
    d = r_i - r_cm #Distance of component from total center of mass

    I_parallel = m * (np.inner(d, d)*I_3 - np.outer(d, d))

    I_total += I_centroids[i] + I_parallel

print("Body frame inertia matrix: ")
print(I_total)

Body frame inertia matrix: 
[[ 89.64564366  -0.910875     0.40429687]
 [ -0.910875   132.52283201   0.68607955]
 [  0.40429687   0.68607955 101.64175956]]


In [25]:
#Calculate Principal Axes (in body coordinates)
eigenvalues, eigenvectors = np.linalg.eigh(I_total)

print("Principal moments:", eigenvalues)
print("Principal axes (columns, in body coords):\n", eigenvectors)

#Sanity check that I_p is diagonal
R = eigenvectors
I_p = R.T @ I_total @ R
print("Ip in principal frame:\n", I_p)

Principal moments: [ 89.61171138 101.64150783 132.55701602]
Principal axes (columns, in body coords):
 [[ 0.99915654 -0.03528101 -0.02101071]
 [ 0.02176583  0.02115719  0.9995392 ]
 [-0.03482022 -0.99915345  0.02190727]]
Ip in principal frame:
 [[ 8.96117114e+01 -9.82440418e-16  1.18243614e-16]
 [-1.07733146e-15  1.01641508e+02  3.52648151e-14]
 [-5.76043840e-16  3.51576130e-14  1.32557016e+02]]


In [26]:
#Discretization

#BUS (hexagonal prism)
s = 2*R_bus/np.sqrt(3) #hexagon side length
A_side = s * H_bus #side rectangle area
A_top = (3*np.sqrt(3)/2) * s**2 #hexagon top/bottom area

print("Hexagonal Prism Bus: ")
print("Side face area:", A_side)
print("Top/Bottom area:", A_top)

print("Side Faces:")
for k in range(6):
    theta = np.deg2rad(60*k)

    n = np.array([np.cos(theta), np.sin(theta), 0.0])   # normal
    r = R_bus * n                                       # centroid

    print(f"Face {k+1}")
    print("centroid:", r)
    print("normal:", n)
    print("area:", A_side)
    print()

print("Top Face")
print("centroid:", [0,0,H_bus/2])
print("normal:", [0,0,1])
print("area:", A_top)

print("Bottom Face")
print("centroid:", [0,0,-H_bus/2])
print("normal:", [0,0,-1])
print("area:", A_top)

Hexagonal Prism Bus: 
Side face area: 0.5398225016923002
Top/Bottom area: 1.0478907385791711
Side Faces:
Face 1
centroid: [0.55 0.   0.  ]
normal: [1. 0. 0.]
area: 0.5398225016923002

Face 2
centroid: [0.275      0.47631397 0.        ]
normal: [0.5       0.8660254 0.       ]
area: 0.5398225016923002

Face 3
centroid: [-0.275       0.47631397  0.        ]
normal: [-0.5        0.8660254  0.       ]
area: 0.5398225016923002

Face 4
centroid: [-5.5000000e-01  6.7355574e-17  0.0000000e+00]
normal: [-1.0000000e+00  1.2246468e-16  0.0000000e+00]
area: 0.5398225016923002

Face 5
centroid: [-0.275      -0.47631397  0.        ]
normal: [-0.5       -0.8660254  0.       ]
area: 0.5398225016923002

Face 6
centroid: [ 0.275      -0.47631397  0.        ]
normal: [ 0.5       -0.8660254  0.       ]
area: 0.5398225016923002

Top Face
centroid: [0, 0, 0.425]
normal: [0, 0, 1]
area: 1.0478907385791711
Bottom Face
centroid: [0, 0, -0.425]
normal: [0, 0, -1]
area: 1.0478907385791711


In [27]:
#Solar panel 
print("Solar Panels")
A_panel = 1.1 * 0.89

print("Panel 1")
print("centroid:", [1.4, 0, 0])
print("normal:", [0, 1, 0])
print("area:", A_panel)

print("Panel 2")
print("centroid:", [-1.4, 0, 0])
print("normal:", [0, 1, 0])
print("area:", A_panel)


#Antenna dish (1 circle)
A_ant = np.pi*(0.35**2)
n_ant = np.array([0.5, 0.8660254, 0.0]) #Antenna is angled 30 deg from +y toward +x axes

print("Antenna Dish")
print(" centroid:", [0.33, 0.56, 0])
print(" normal:", n_ant)
print(" area:", A_ant)

Solar Panels
Panel 1
centroid: [1.4, 0, 0]
normal: [0, 1, 0]
area: 0.9790000000000001
Panel 2
centroid: [-1.4, 0, 0]
normal: [0, 1, 0]
area: 0.9790000000000001
Antenna Dish
 centroid: [0.33, 0.56, 0]
 normal: [0.5       0.8660254 0.       ]
 area: 0.3848451000647496
